In [7]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, d199589f-b2ca-4cd0-b566-e0e41eb5774c, 9, Finished, Available, Finished)

In [8]:
%%sql
CREATE TABLE IF NOT EXISTS Ecommere_Lh.gold_Customer
(
    Customer_ID string  ,
    Customer_Name string    ,
    Segment string  ,
    City    string  ,
    State string    ,
    Country string  ,
    Region string   ,
    Created_TS timestamp    ,
    Modified_TS timestamp
)
USING DELTA

StatementMeta(, d199589f-b2ca-4cd0-b566-e0e41eb5774c, 10, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [9]:
max_date = spark.sql("""SELECT coalesce(max("Modified_TS"),"1900-01-01") FROM Ecommere_Lh.gold_customer""").first()[0]

StatementMeta(, d199589f-b2ca-4cd0-b566-e0e41eb5774c, 11, Finished, Available, Finished)

In [10]:
max_date

StatementMeta(, d199589f-b2ca-4cd0-b566-e0e41eb5774c, 12, Finished, Available, Finished)

'1900-01-01'

In [11]:
df_bronze = spark.read.table("Ecommere_Lh.bronze_sales")
df_final = df_bronze.selectExpr("Customer_ID", "Customer_Name",\
                "Segment", "city", "State", "Country", "Region") \
                .where(col("Modified_TS")>max_date)\
                .drop_duplicates()
                

StatementMeta(, d199589f-b2ca-4cd0-b566-e0e41eb5774c, 13, Finished, Available, Finished)

In [13]:
df_final.createOrReplaceTempView("ViewCustomer")

StatementMeta(, d199589f-b2ca-4cd0-b566-e0e41eb5774c, 15, Finished, Available, Finished)

In [14]:
%%sql
SELECT * FROM ViewCustomer

StatementMeta(, d199589f-b2ca-4cd0-b566-e0e41eb5774c, 16, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 7 fields>

In [17]:
%%sql
MERGE INTO Ecommere_Lh.gold_customer as gc
USING ViewCustomer as vc
on gc.Customer_ID = vc.Customer_ID
WHEN MATCHED THEN
UPDATE SET
gc.Customer_Name = vc.Customer_Name ,
gc.Segment = vc.Segment ,
gc.City = vc.City   ,
gc.State  =  vc.State    ,
gc.Country = vc.Country ,
gc.Region = vc.Region   ,
gc.Modified_TS = CURRENT_TIMESTAMP()

WHEN NOT MATCHED THEN INSERT
(
gc.Customer_ID,
gc.Customer_Name  ,
gc.Segment ,
gc.City  ,
gc.State    ,
gc.Country ,
gc.Region    ,
gc.Created_TS   ,
gc.Modified_TS
)
VALUES
(
vc.Customer_ID,
vc.Customer_Name  ,
vc.Segment ,
vc.City  ,
vc.State    ,
vc.Country ,
vc.Region    ,
CURRENT_TIMESTAMP(),
CURRENT_TIMESTAMP
)

StatementMeta(, d199589f-b2ca-4cd0-b566-e0e41eb5774c, 19, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [1]:
%%sql
SELECT * FROM gold_Customer LIMIT 10

StatementMeta(, 046259d5-410a-400b-a546-2b7d391f5f57, 2, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 9 fields>